In [100]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,f1_score

In [3]:
df = pd.read_excel('sdbi_proxy_bad_def_dataset.xlsx')

In [4]:
df_copy = df.copy()

In [5]:
df.shape

(172929, 3)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172929 entries, 0 to 172928
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   id      172929 non-null  int64
 1   mob     172929 non-null  int64
 2   dpd     172929 non-null  int64
dtypes: int64(3)
memory usage: 4.0 MB


In [6]:
df.head(3)

,id,mob,dpd
0,1,0,0
1,1,1,0
2,1,2,0


#### Unique ids

In [8]:
df.id.nunique()

6999

In [31]:
## Case 1: Some months can be missing
all_months = list(range(0,25))
rem_ids = []
for i in df.id.unique():
    if ((list(df.query('id == {}'.format(i))['mob'].values) == all_months) or (list(df.query('id == {}'.format(i))['mob'].values) == all_months[1:])):
        continue
    else:
        print('id ',i,' has some missing months')
        rem_ids.append(i)

id  5253  has some missing months
id  5292  has some missing months


#### Dropping ids which do not have 24 months information

In [34]:
for i in rem_ids:
    df.drop(df[df['id'] == i].index,inplace=True)

# Eventual bad definition:

- Ever 90+ in first 24 months of repayment then "BAD"

In [35]:
df['max_dpd'] = df.groupby('id').dpd.max()[df.id].reset_index().dpd

In [43]:
grouped_df = df.groupby('id')[['dpd']].max()

In [46]:
grouped_df['eventual_bad'] = np.where(grouped_df['dpd'] >= 90,1,0)

In [54]:
df['eventual_bad'] = np.where(df['max_dpd']>=90,1,0)

In [48]:
grouped_df.head(15)

,dpd,eventual_bad
id,,
1,0,0
2,28,0
3,0,0
4,0,0
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0


#### removing records older than 12 months

In [69]:
df_ = df_copy.drop(df[df['mob'] > 12].index)

In [70]:
df_['max_dpd'] = df_.groupby('id').dpd.max()[df_.id].reset_index().dpd

In [90]:
grouped_df_ = df_.groupby('id')[['dpd']].max()

In [92]:
grouped_df_['proxy_bad'] = np.where(grouped_df_['dpd']>0,1,0)

In [93]:
grouped_df_.head(14)

,dpd,proxy_bad
id,,
1,0,0
2,0,0
3,0,0
4,0,0
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0


In [94]:
grouped_df.head(14)

,dpd,eventual_bad
id,,
1,0,0
2,28,0
3,0,0
4,0,0
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0


In [97]:
merged_df = pd.merge(grouped_df,grouped_df_,left_index=True,right_index=True)

In [98]:
merged_df.query('id == 57')

,dpd_x,eventual_bad,dpd_y,proxy_bad
id,,,,
57,93,1,0,0


In [101]:
confusion_matrix(merged_df['eventual_bad'],merged_df['proxy_bad'])

array([[6513,  250],
       [ 114,  120]])

In [102]:
accuracy_score(merged_df['eventual_bad'],merged_df['proxy_bad'])

0.9479777047305988

In [103]:
precision_score(merged_df['eventual_bad'],merged_df['proxy_bad'])

0.32432432432432434

In [104]:
recall_score(merged_df['eventual_bad'],merged_df['proxy_bad'])

0.5128205128205128

In [105]:
f1_score(merged_df['eventual_bad'],merged_df['proxy_bad'])

0.3973509933774834